In [2]:
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

In [4]:
df = pd.read_csv("Recommendation_yoga_data.csv")
df.head()

,Unnamed: 0,UserID,Mood Before,Yoga Practice,Duration,Frequency,Mood After,Feedback
0,0,1,Fear,Parighasana,30 min,Weekly,Angry,No significant change
1,1,2,Angry,Salabhasana,60 min,Daily,Jealousy,Felt more relaxed
2,2,3,Addiction,Vajrasana,15 min,Monthly,Overwhelmed,Improved mood
3,3,4,Gratitude,Parsva Bakasana,60 min,Twice a week,Loneliness,Improved mood
4,4,5,Angry,Ardha Pincha Mayurasana,30 min,Weekly,Contentment,Helped with anxiety


In [7]:
moods = df['Mood Before'].unique()
poses = df['Yoga Practice'].unique()
mood_to_idx = {mood: idx for idx, mood in enumerate(moods)}
pose_to_idx = {pose: idx for idx, pose in enumerate(poses)}


In [19]:
interaction_matrix = csr_matrix((len(mood_to_idx), len(pose_to_idx)), dtype=int)
interaction_matrix_float = interaction_matrix.astype('float32')

In [29]:
for _, row in df.iterrows():
    mood_idx = mood_to_idx[row['Mood Before']]
    pose_idx = pose_to_idx[row['Yoga Practice']]
    interaction_matrix_float[mood_idx, pose_idx] += 1

num_features = min(interaction_matrix_float.shape) - 1  # Choose a suitable number of features
u, sigma, vt = svds(interaction_matrix_float, k=num_features)
sigma_diag_matrix = np.diag(sigma)

In [38]:
def recommend_poses(mood, u, sigma, vt, top_n=3):
    mood_idx = mood_to_idx[mood]
    mood_latent_features = np.dot(u[mood_idx, :], sigma)
    pose_scores = np.dot(mood_latent_features, vt)

    top_pose_indices = np.argsort(pose_scores)[::-1][:top_n]
    recommended_poses = [poses[idx] for idx in top_pose_indices]

    return recommended_poses

['Chakravakasana', 'Astavakrasana', 'Agnistambhasana']


In [ ]:
recommended_poses = recommend_poses('Angry', u, sigma_diag_matrix, vt, top_n=3)
print(recommended_poses)